# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,touros,-5.1989,-35.4608,26.75,83,73,2.91,BR,1709350645
1,1,olinda,-8.0089,-34.8553,27.85,89,20,2.06,BR,1709350646
2,2,miracema do tocantins,-9.5618,-48.3967,24.21,92,31,1.07,BR,1709350646
3,3,badger,64.8000,-147.5333,-24.07,52,0,0.00,US,1709350490
4,4,ancud,-41.8697,-73.8203,11.90,86,3,2.59,CL,1709350482


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [7]:
%%capture --no-display

# Configure the map plot
cities_map = city_data_df.hvplot.points(x="Lng", y="Lat", geo=True, tiles="OSM", hover_cols=["Humidity"],
                                                color="City", alpha=0.5, size=50, width=1000, height=600)

# Display the map
cities_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [20]:
# Narrow down cities that fit criteria and drop any results with null values
min_temp = 25
max_temp = 35
min_humidity = 20
max_humidity = 60
max_wind_speed = 5
max_cloudiness = 50

selected_city_df = city_data_df[(city_data_df["Max Temp"] >= min_temp) & (city_data_df["Max Temp"] <= max_temp) &
                                (city_data_df["Humidity"] >= min_humidity) & (city_data_df["Humidity"] <= max_humidity) &
                                (city_data_df["Wind Speed"] <= max_wind_speed) & (city_data_df["Cloudiness"] <= max_cloudiness)]


# Drop any rows with null values
selected_city_df = selected_city_df.dropna()

# Display sample data
selected_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
8,8,gambela,8.2500,34.5833,25.71,51,46,1.42,ET,1709350647
30,30,yanam,16.7333,82.2167,31.54,42,3,0.82,IN,1709350652
40,40,phulbani,20.4667,84.2333,30.63,21,0,1.16,IN,1709350653
52,52,margaret river,-33.9500,115.0667,27.19,47,17,3.55,AU,1709350490
103,103,noosa heads,-26.3833,153.1500,32.41,56,0,4.35,AU,1709350663
208,208,tabuk,17.4189,121.4443,28.46,56,40,1.74,PH,1709350679
258,258,emerald,-23.5333,148.1667,32.97,35,0,4.23,AU,1709350687
265,265,malvan,16.0667,73.4667,27.44,58,46,0.43,IN,1709350688
390,390,pawai,24.2667,80.1667,28.26,27,37,2.93,IN,1709350710
430,430,camapua,-19.5314,-54.0439,26.30,53,21,1.23,BR,1709350716


### Step 3: Create a new DataFrame called `hotel_df`.

In [24]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = selected_city_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
8,gambela,ET,8.2500,34.5833,51,
30,yanam,IN,16.7333,82.2167,42,
40,phulbani,IN,20.4667,84.2333,21,
52,margaret river,AU,-33.9500,115.0667,47,
103,noosa heads,AU,-26.3833,153.1500,56,
208,tabuk,PH,17.4189,121.4443,56,
258,emerald,AU,-23.5333,148.1667,35,
265,malvan,IN,16.0667,73.4667,58,
390,pawai,IN,24.2667,80.1667,27,
430,camapua,BR,-19.5314,-54.0439,53,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [27]:
# Set parameters to search for a hotel
radius = 20000
params = {
    "categories": "accommodation",
    "apiKey": geoapify_key,
    "format": "json"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
gambela - nearest hotel: Giwa hotel
yanam - nearest hotel: Hôtel Sarvainn
phulbani - nearest hotel: No hotel found
margaret river - nearest hotel: No hotel found
noosa heads - nearest hotel: Costa Nova Apartments
tabuk - nearest hotel: Davidson Hotel
emerald - nearest hotel: Emerald Executive Apartments
malvan - nearest hotel: Hotel Silver Sand
pawai - nearest hotel: No hotel found
camapua - nearest hotel: No hotel found
lakshmipur - nearest hotel: লক্ষ্মীপুর সার্কিট হাউস
konarka - nearest hotel: Hotel Sun Village
feni - nearest hotel: হোটেল নাইট গেস্ট
kuchaiburi - nearest hotel: No hotel found
bhamo - nearest hotel: Winlight Hotel


,City,Country,Lat,Lng,Humidity,Hotel Name
8,gambela,ET,8.2500,34.5833,51,Giwa hotel
30,yanam,IN,16.7333,82.2167,42,Hôtel Sarvainn
40,phulbani,IN,20.4667,84.2333,21,No hotel found
52,margaret river,AU,-33.9500,115.0667,47,No hotel found
103,noosa heads,AU,-26.3833,153.1500,56,Costa Nova Apartments
208,tabuk,PH,17.4189,121.4443,56,Davidson Hotel
258,emerald,AU,-23.5333,148.1667,35,Emerald Executive Apartments
265,malvan,IN,16.0667,73.4667,58,Hotel Silver Sand
390,pawai,IN,24.2667,80.1667,27,No hotel found
430,camapua,BR,-19.5314,-54.0439,53,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [28]:
%%capture --no-display

# Configure the map plot
hotels_map = hotel_df.hvplot.points(x="Lng", y="Lat", geo=True, tiles="OSM", hover_cols=["Humidity", "Hotel Name", "Country"],
                                    color="City", alpha=0.5, size=50, width=1000, height=600)

# Display the map
hotels_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)